In [6]:
#importing module from library keras
from keras.models import load_model
#importing 2 libraries: cv2 and numpy as np
import cv2
import numpy as np

In [7]:
#loading the best model we got after training and testing
model = load_model('model-017.model')

#getting the region of intrest from video using cascade classifier
face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#using default camera to get input data
source=cv2.VideoCapture(0)

#creating dictionary for mask and no mask
labels_dict={0:'MASK',1:'NO MASK'}

#creating dictionary for red and green boxes
color_dict={0:(0,255,0),1:(0,0,255)}

In [8]:
#running an infinite loop
while(True):

    #getting video input
    ret,img=source.read()
    
    #grayscalling the video input
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    #detecting face
    faces=face_clsfr.detectMultiScale(gray,1.3,5)  

    for x,y,w,h in faces:
    
        #for every frame taking out region of intrest and storing in variable resized
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        
        #normalizing the input
        normalized=resized/255.0
        
        #reshaping the input to 4D for NN to work on
        reshaped=np.reshape(normalized,(1,100,100,1))
        
        #predicting wether person is wearing mask or not
        result=model.predict(reshaped)

        #storing the result wether person is wearing mask or not in variable label
        label=np.argmax(result,axis=1)[0]
      
        #displaying rectangle around the ROI with the label
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    #giving the live output
    cv2.imshow('LIVE',img)
    
    #registring keyboard input if any
    key=cv2.waitKey(1)
    
    #checking if esc key is pressed to terminate the camera window
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()